In [10]:
# pip install psychopy
# pip install psychopy --no-deps

In [1]:
from psychopy import visual
import pandas as pd
from PIL import Image
import time

In [2]:
images_map_path = 'Data/imagemap.xlsx'
features_path = 'Data/testfeature.xlsx'

In [3]:
images_file = pd.read_excel(images_map_path, index_col=0)
images_file.fillna(0, inplace=True)
images_file.head()

,Image Path,Arms_X,Arms_Y,Legs_X,Legs_Y,Eyes_X,Eyes_Y,Mouth_X,Mouth_Y,Antenna_X,Antenna_Y,Tail_X,Tail_Y,Misc_X,Misc_Y
ID,,,,,,,,,,,,,,,
body_1,Data/Images/body_1.png,0.0,-0.09,0.014,-0.37,0.015,0.17,-0.01,-0.01,0.0,0.33,0.017,0.15,0.003,-0.152
body_2,Data/Images/body_2.png,0.0,0.00,0.000,0.00,0.000,0.00,0.00,0.00,0.0,0.00,0.000,0.00,0.000,0.000
1_arms_1,Data/Images/1_arms_1.png,0.0,0.00,0.000,0.00,0.000,0.00,0.00,0.00,0.0,0.00,0.000,0.00,0.000,0.000
1_arms_2,Data/Images/1_arms_2.png,0.0,0.00,0.000,0.00,0.000,0.00,0.00,0.00,0.0,0.00,0.000,0.00,0.000,0.000
1_arms_3,Data/Images/1_arms_3.png,0.0,0.00,0.000,0.00,0.000,0.00,0.00,0.00,0.0,0.00,0.000,0.00,0.000,0.000


In [4]:
features_file = pd.read_excel(features_path)
features_file.head()

,Phase,Schedule,Context,Side,Body,Arms,Legs,Eyes,Mouth,Antenna,Tail,Color
0,study,blocked,context1,right,1,3,4,1,2,4,2,100
1,test-memory,interleaved,context2,left,1,1,2,2,1,1,3,200
2,test-feature,interleaved,context3,right,1,4,1,4,1,2,3,300


In [62]:
def get_aliens(images_map_path, features_path, window):
    """
    Returns an array of list of stimuli for each alien.
    """
    
    images_file = pd.read_excel(images_map_path, index_col=0)
    images_file.fillna(0, inplace=True)
    features_file = pd.read_excel(features_path)
    
    aliens = []
    
    size = 0.3
    
    for index, row in features_file.iterrows():
        color = row['Color']
        
        body_id = 'body_' + str(row['Body'])
        body_row = images_file.loc[body_id]
        path_col = images_file['Image Path']
        
        side_offset = 0.5 if row['Side'] == 'right' else -0.5
        
        # Body
        path = path_col.loc[body_id]
        pos = (side_offset, 0)
        body = visual.ImageStim(window, image=path, pos=pos, color=color, colorSpace='rgb255')
        body.size *= size
        
        base_id = str(row['Body']) + '_{}_{}'
        
        # Arms
        path = path_col.loc[base_id.format('arms', row['Arms'])]
        pos = (body_row['Arms_X'] + side_offset, body_row['Arms_Y'])
        arms = visual.ImageStim(window, image=path, pos=pos, color=color, colorSpace='rgb255')
        arms.size *= size
        
        # Legs
        path = path_col.loc[base_id.format('legs', row['Legs'])]
        pos = (body_row['Legs_X'] + side_offset, body_row['Legs_Y'])
        legs = visual.ImageStim(window, image=path, pos=pos)
        legs.size *= size
        
        # Eyes
        path = path_col.loc[base_id.format('eyes', row['Eyes'])]
        pos = (body_row['Eyes_X'] + side_offset, body_row['Eyes_Y'])
        eyes = visual.ImageStim(window, image=path, pos=pos)
        eyes.size *= size
        
        # Mouth
        path = path_col.loc[base_id.format('mouth', row['Mouth'])]
        pos = (body_row['Mouth_X'] + side_offset, body_row['Mouth_Y'])
        mouth = visual.ImageStim(window, image=path, pos=pos)
        mouth.size *= size
        
        # Antenna
        path = path_col.loc[base_id.format('antenna', row['Antenna'])]
        pos = (body_row['Antenna_X'] + side_offset, body_row['Antenna_Y'])
        antenna = visual.ImageStim(window, image=path, pos=pos)
        antenna.size *= size
        
        # Tail
        path = path_col.loc[base_id.format('tail', row['Tail'])]
        pos = (body_row['Tail_X'] + side_offset, body_row['Tail_Y'])
        pos = (side_offset + 0.017, -0.15)
        tail = visual.ImageStim(window, image=path, pos=pos)
        tail.size *= size
        
        # Misc (depends on body)
        gloves = None
        if row['Body'] == 1:
            # Gloves
            path = path_col.loc[base_id.format('gloves', row['Arms'])]
            pos = (body_row['Misc_X'] + side_offset, body_row['Misc_Y'])
            gloves = visual.ImageStim(window, image=path, pos=pos)
            gloves.size *= size
        
        stim_list = [arms, legs, body, eyes, mouth, antenna, tail]
        if gloves:
            stim_list.append(gloves)
        
        aliens.append(stim_list)
        
    return aliens

In [63]:
window = visual.Window([1400, 900], monitor='testMonitor')

try:
    aliens = get_aliens(images_map_path, features_path, window)

    screenshot = visual.BufferImageStim(window, stim=aliens[0])

    screenshot.draw()
    
except:
    pass

window.flip()

time.sleep(8)

window.close()